## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import config from another folder
import sys
sys.path.append('../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [13]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Avarua,CK,-21.2078,-159.7750,86.05,74,25,9.22,scattered clouds,2022-03-31 02:14:06
1,1,Upernavik,GL,72.7868,-56.1549,13.66,84,36,6.44,scattered clouds,2022-03-31 02:14:06
2,2,Atuona,PF,-9.8000,-139.0333,79.07,72,29,12.80,scattered clouds,2022-03-31 02:14:07
3,3,Vyshchetarasivka,UA,47.5685,34.8810,44.98,83,100,11.41,overcast clouds,2022-03-31 02:14:07
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,-15.48,94,100,8.34,overcast clouds,2022-03-31 02:14:08


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Avarua,CK,-21.2078,-159.7750,86.05,74,25,9.22,scattered clouds,2022-03-31 02:14:06
2,2,Atuona,PF,-9.8000,-139.0333,79.07,72,29,12.80,scattered clouds,2022-03-31 02:14:07
16,16,Rikitea,PF,-23.1203,-134.9692,78.37,77,97,16.82,light rain,2022-03-31 02:09:29
17,17,Acarau,BR,-2.8856,-40.1200,78.44,83,100,9.13,overcast clouds,2022-03-31 02:14:13
19,19,Coolum Beach,AU,-26.5333,153.1000,82.35,68,52,1.01,broken clouds,2022-03-31 02:14:14
28,28,Victoria,HK,22.2855,114.1577,82.45,69,2,11.99,clear sky,2022-03-31 02:11:12
30,30,Bonthe,SL,7.5264,-12.5050,78.17,93,7,5.21,clear sky,2022-03-31 02:14:18
32,32,Arraial Do Cabo,BR,-22.9661,-42.0278,76.48,91,0,13.11,clear sky,2022-03-31 02:14:19
36,36,Busselton,AU,-33.6500,115.3333,75.36,58,45,15.41,scattered clouds,2022-03-31 02:14:20
41,41,Puerto Ayora,EC,-0.7393,-90.3518,75.54,97,100,5.99,moderate rain,2022-03-31 02:14:22


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                195
City                   195
Country                195
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
Date                   195
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,86.05,scattered clouds,-21.2078,-159.7750,
2,Atuona,PF,79.07,scattered clouds,-9.8000,-139.0333,
16,Rikitea,PF,78.37,light rain,-23.1203,-134.9692,
17,Acarau,BR,78.44,overcast clouds,-2.8856,-40.1200,
19,Coolum Beach,AU,82.35,broken clouds,-26.5333,153.1000,
28,Victoria,HK,82.45,clear sky,22.2855,114.1577,
30,Bonthe,SL,78.17,clear sky,7.5264,-12.5050,
32,Arraial Do Cabo,BR,76.48,clear sky,-22.9661,-42.0278,
36,Busselton,AU,75.36,scattered clouds,-33.6500,115.3333,
41,Puerto Ayora,EC,75.54,moderate rain,-0.7393,-90.3518,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,86.05,scattered clouds,-21.2078,-159.7750,Paradise Inn
2,Atuona,PF,79.07,scattered clouds,-9.8000,-139.0333,Villa Enata
16,Rikitea,PF,78.37,light rain,-23.1203,-134.9692,People ThankYou
17,Acarau,BR,78.44,overcast clouds,-2.8856,-40.1200,castelo pizzaria
19,Coolum Beach,AU,82.35,broken clouds,-26.5333,153.1000,Coolum Beach Resort
28,Victoria,HK,82.45,clear sky,22.2855,114.1577,Mini Hotel Central
30,Bonthe,SL,78.17,clear sky,7.5264,-12.5050,Bonthe Holiday Village
32,Arraial Do Cabo,BR,76.48,clear sky,-22.9661,-42.0278,Pousada Porto Praia
36,Busselton,AU,75.36,scattered clouds,-33.6500,115.3333,Observatory Guest House
41,Puerto Ayora,EC,75.54,moderate rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
        <dl>
        <dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
        <dt><b>City</b></dt><dd>{City}</dd>
        <dt><b>Country</b></dt><dd>{Country}</dd>
        <dt><b>Weather description</b></dt><dd>{Current Description} and max {Max Temp} °F</dd>
        </dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
list_locations = list(locations.to_records(index=False))

In [32]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))